In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

--2022-04-18 10:43:42--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-18 10:43:42--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   112MB/s    in 0.6s    

2022-04-18 10:43:43 (112 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
import numpy as np

level = 'hf/3-21G'

mol: psi4.core.Molecule = psi4.geometry('''
0 1
 C   -1.073986885437   -1.170578359942    0.000002755738
 C    0.953021017198    0.140745143377   -0.000003061297
 C    0.196029366481    1.360797354321   -0.000001202096
 C   -1.155622586106    1.236381947214    0.000002880952
 C   -1.818434996700   -0.031125507024    0.000005467599
 H   -1.510894241728   -2.166796480067    0.000004269828
 H    0.691726656323    2.319296967258   -0.000002742435
 H   -1.770734621411    2.131633416218    0.000004678262
 H   -2.895472126141   -0.076640107909    0.000009377672
 O    2.177846935050   -0.264655641303   -0.000003962413
 H    1.432324292904   -1.412270186206   -0.000002894660
 N    0.288380859812   -1.071403950594   -0.000002249524
''')

print(f'optimizing TS w/ water ...')
psi4.set_options({'opt_type': 'ts',
                  'full_hess_every': 0,
                  'geom_maxiter': 300})

_, wfn = psi4.optimize(level,
                       molecule=mol,
                       return_wfn=True)

print(f'calculating frequencies ...')
_, wfn = psi4.frequency(level,
                        molecule=mol,
                        ref_gradient=wfn.gradient(),
                        return_wfn=True)

print(f'optimized structure:\n{mol.save_string_xyz()}')
freqs = np.array(wfn.frequencies())
print(f'frequencies:\n{freqs}')

optimizing TS w/ water ...
Optimizer: Optimization complete!
calculating frequencies ...
optimized structure:
0 1
 C   -1.058491888914   -1.160969921328    0.000001354899
 C    0.929008468290    0.120368081764   -0.000002986433
 C    0.212138872601    1.317561248800   -0.000002445318
 C   -1.156725513965    1.219756387816    0.000001737720
 C   -1.818880339809   -0.020004964336    0.000004387699
 H   -1.485804052494   -2.141073257311    0.000003708834
 H    0.720665245122    2.255563067182   -0.000006697237
 H   -1.742065811162    2.118076950828    0.000002160450
 H   -2.885795461334   -0.074125149254    0.000008428112
 O    2.180185349054   -0.227553913090    0.000001337676
 H    1.523345905913   -1.414735811300   -0.000012580735
 N    0.267331348104   -1.059078163742   -0.000002925020

frequencies:
[-2230.61783112   248.31197742   482.21567652   529.94396032
   556.96615424   672.50721272   741.99792394   847.10015334
   891.84492859   947.83981297  1007.0615551   1098.53014212
  111

In [4]:
psi4.core.set_output_file("H2O.out")

mol = psi4.geometry("""
symmetry c1
0 1
O       -1.0239104333      1.3152089264     -0.0149071027                 
H       -0.0345840622      1.2807129464      0.0105245830                 
H       -1.3132760473      0.5529382075      0.5470664770                 
""")

# input (PCM Solver)
pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = Water
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
}
'''
psi4.set_options({'pcm': True})
psi4.pcm_helper(pcm)

scf_e, wfn = psi4.energy("B3LYP/6-31G", return_wfn = True)
print('energy at the {} level of theory:\t{:.4f}'.format("B3LYP/6-31G", scf_e))

energy at the B3LYP/6-31G level of theory:	-76.3928


In [5]:
level = 'B3LYP/6-31G(d)'

pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = Water
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
}
'''

ts_water = psi4.geometry('''
0 1
 C   -2.059804939004   -0.883939515300    0.016896641594
 C   -2.142670896585    0.522601087867    0.034373401662
 C   -1.005327856892    1.292137964650    0.025819124801
 C    0.262128691322    0.655780273599    0.000149989930
 C   -0.809296733651   -1.450233360684   -0.006977337441
 H   -2.947090445439   -1.504862009531    0.020116162769
 H   -3.116351053856    1.004058321974    0.053181708105
 H   -1.034287493258    2.374975753357    0.035605898574
 H   -0.659869391828   -2.525258880890   -0.023896123570
 N    0.310925009160   -0.705786602154   -0.011754036656
 O    1.373676571106    1.306595996626   -0.012556673040
 H    2.241663568347    0.464742550392   -0.051033211027
 H    1.539325665427   -1.010395904629   -0.033586558790
 O    2.724571979738   -0.663301399386   -0.078932817322
 H    3.137792164668   -0.829819474500    0.778335970279
''')

psi4.set_options({'pcm': True})
psi4.pcm_helper(pcm)

print(f'calc energy of TS w/ explicit water molecule ...')
e = psi4.energy(level,
                molecule=ts_water)
print(f'calcd energy: {e}')

calc energy of TS w/ explicit water molecule ...
calcd energy: -399.934308461846
